In [1]:
import pandas as pd
import pickle
print('loading...')
with open('article_objects_noempty.pkl', 'rb') as file_in:
    text_objects = pickle.load(file_in)
df = pd.DataFrame(text_objects)
text_data = df['cleaned_text'].to_list()

loading...


In [2]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from unidecode import unidecode
import re
from tqdm.notebook import tqdm

stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
cachedStopWords = stopwords.words("english")

def preprocess(raw):
    result = unidecode(raw)
    result = re.sub(r'(https?:\/\/)?([a-zA-Z0-9-_]{1,}\.){1,}[a-zA-Z0-9-_]{1,}(\/[A-Za-z0-9-._~:?#\[\]@!$&\'()*+,;%=]{1,}){0,}\/?', '', result)
    result = re.sub(r"[^a-zA-Z]", " ", result)
    result = re.sub(' +', ' ', result)
    result = result.lower()
    result = stemmer.stem(result)
    result = [lemmatiser.lemmatize(w) for w in result.split(' ')]
    result = [w for w in result if w not in cachedStopWords]
    result = [w for w in result if len(w) > 4]
    return result

tp = []
for article in tqdm(text_data):
    preprocessed_article = preprocess(article)
    if len(preprocessed_article) >= 40:
        tp.append(preprocessed_article)

In [4]:
len(tp)

4138

In [5]:
import random
random.shuffle(tp)

tp_train = tp[500:]
tp_test = tp[:500]

In [6]:
import gensim

dictionary = gensim.corpora.Dictionary(tp_train)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in tp]

In [7]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [11]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

In [12]:
for idx, topic in lda_model_tfidf.print_topics(num_words=10):
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.005*"county" + 0.003*"china" + 0.003*"death" + 0.002*"california" + 0.002*"patient" + 0.002*"official" + 0.002*"reported" + 0.002*"friday" + 0.002*"positive" + 0.002*"total"

Topic: 1 Word: 0.004*"county" + 0.002*"patient" + 0.002*"death" + 0.002*"reported" + 0.002*"california" + 0.002*"symptom" + 0.002*"country" + 0.001*"official" + 0.001*"testing" + 0.001*"mutation"

Topic: 2 Word: 0.004*"vaccine" + 0.003*"china" + 0.002*"country" + 0.002*"death" + 0.002*"trump" + 0.002*"company" + 0.002*"chinese" + 0.002*"government" + 0.002*"patient" + 0.002*"county"

Topic: 3 Word: 0.003*"county" + 0.003*"death" + 0.002*"china" + 0.002*"patient" + 0.002*"vaccine" + 0.002*"positive" + 0.002*"antibody" + 0.002*"total" + 0.002*"country" + 0.002*"trump"

Topic: 4 Word: 0.005*"vaccine" + 0.004*"antibody" + 0.003*"trial" + 0.003*"patient" + 0.002*"study" + 0.002*"company" + 0.002*"treatment" + 0.002*"immune" + 0.002*"moderna" + 0.002*"phase"

Topic: 5 Word: 0.003*"patient" + 0.003*"plas

In [13]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

vis = pyLDAvis.gensim.prepare(lda_model_tfidf, corpus_tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.021632  0.001432       1        1  52.654900
4      0.021363 -0.005766       2        1  17.450094
7     -0.001048  0.012223       3        1  13.136294
9      0.015480  0.003451       4        1   5.589944
2      0.003427  0.022965       5        1   3.183774
5      0.003450 -0.015155       6        1   1.910145
8     -0.008000 -0.008143       7        1   1.792859
3     -0.006589  0.002839       8        1   1.560928
1     -0.015185 -0.006757       9        1   1.384144
6      0.008734 -0.007090      10        1   1.336914, topic_info=            Term        Freq       Total Category  logprob  loglift
686       county  138.000000  138.000000  Default  30.0000  30.0000
142        death   89.000000   89.000000  Default  29.0000  29.0000
590      patient   87.000000   87.000000  Default  28.0000  28.0000
1377     vaccine   93.000000   93.000000  Default  27.0000  27.0000
429        china   96.000000   96.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
88        public    0.614870   49.984791  Topic10  -6.7778  -0.0833
1153   confirmed    0.619192   52.875240  Topic10  -6.7708  -0.1325
447   government    0.618884   54.725868  Topic10  -6.7713  -0.1674
571     lockdown    0.602694   43.931965  Topic10  -6.7978   0.0258
163     reported    0.601516   60.499687  Topic10  -6.7997  -0.2961

[793 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5192      2  0.583739     abbvie
130       1  0.579899  according
130       2  0.156127  according
130       3  0.111519  according
130       4  0.044608  according
...     ...       ...        ...
5173      4  0.449398      yemen
2882      3  0.565646     yunnan
2539      1  0.357191    zealand
2539      3  0.178596    zealand
2539      4  0.178596    zealand

[1634 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 8, 10, 3, 6, 9, 4, 2, 7])

In [34]:
from gensim.test.utils import datapath

lda_model_tfidf.save('lda.model')

In [36]:
import numpy
numpy.__version__

'1.18.5'

In [18]:
lda = gensim.models.LdaMulticore.load(temp_file)

In [20]:
tp_test[0]

['first',
 'since',
 'began',
 'reporting',
 'january',
 'chinese',
 'authority',
 'registered',
 'infection',
 'coronavirus',
 'brazil',
 'confirmed',
 'total',
 'coronavirus',
 'overtaking',
 'russia',
 'become',
 'world',
 'second',
 'hotspot',
 'covid',
 'million',
 'infant',
 'could',
 'vaccine',
 'preventable',
 'disease',
 'diphtheria',
 'measles',
 'polio',
 'disruption',
 'routine',
 'immunisation',
 'caused',
 'pandemic',
 'agency',
 'million',
 'people',
 'around',
 'world',
 'confirmed',
 'coronavirus',
 'according',
 'compiled',
 'hopkins',
 'university',
 'people',
 'globally',
 'million',
 'people',
 'recovered',
 'latest',
 'update',
 'woman',
 'raised',
 'question',
 'florida',
 'covid',
 'removed',
 'curator',
 'reprimanded',
 'several',
 'violating',
 'health',
 'department',
 'policy',
 'including',
 'posting',
 'political',
 'commentary',
 'information',
 'state',
 'record',
 'rebekah',
 'jones',
 'comment',
 'email',
 'researcher',
 'interview',
 'handful',
 'medi

In [30]:
test_corpus = [dictionary.doc2bow(doc) for doc in tp_test]
sorted(lda[test_corpus[8]], key=lambda x:-x[1])

[(4, 0.35516104),
 (7, 0.20677634),
 (0, 0.19640242),
 (8, 0.1613356),
 (9, 0.07622183)]

In [46]:
x = [[a, float(b)] for a, b in lda[test_corpus[8]]]
y = [[a, float(b)] for a, b in lda[test_corpus[9]]]

In [47]:
import time
ts1 = time.time()
for i in range(100 * 100):
    sim = gensim.matutils.cossim(a, b)
print(time.time() - ts1)
print(sim)

0.19605278968811035
0.4414360460028187


In [38]:
sim

0.3967043563884227